**DownLoad-InSAR** : Download ASF Hyp3/GAMMA InSAR products from ASF/Vertex by accesss Hyp3 processing service. Downloaded products will be put in the specfied directory within GoogleDrive named WORKING_DIR and JOB_NAME><br>
<pre>Jan,2018 : Isara Chaowuttisuk (Tle) Phichaowuttisuk@hotmail.com
         : Phisan Santitamnont (phisan.chula@gmail.com)
Reference :
1. [hyp3-docs](https://hyp3-docs.asf.alaska.edu/using/sdk/)
2. [asf-search](https://search.asf.alaska.edu/#/?searchType=Custom%20Products&resultsLoaded=true&granule=e514b1d1-f848-47c6-8071-c28056bc4974) <bre>

**1. Mount GoogleCoLab to user's GoogleDrive and define NETRC_CRED, WORKING_DIR, JOB_NAME, DEBUG_PROD in section 4.**

In [2]:
# mount GoogleCoLab to user's GoogleDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**2.Build InSAR/Hyp3 VM :** take some minutes...

In [3]:
%%capture
conda_path = ''
try:
    conda_path = !which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init 
    !conda install -n root _license -y -q
else:
    print('found miniconda')

conda_envs = !conda env list
res = [i for i in conda_envs if 'InSAR' in i]
if (len(res) == 0):
    print('not found InSAR env', len(res))
    !conda create -y -q --name InSAR python=3.9
else:
    print('found InSAR env', len(res))

In [4]:
%%capture
%%bash
source activate InSAR
conda install -c conda-forge asf_search -y
conda install -c conda-forge hyp3_sdk asf_search pandas geopandas -y

python
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages')

print("Python version", sys.version)

**4. Create Job_Download function on InSAR VM :** modifty your NETRC_CRED, WORKING_DIR, JOB_NAME, DEBUG_PROD accordingly...

In [5]:
%%writefile job_download.py
#####################################################################
NETRC_CRED  = '/content/drive/MyDrive/.netrc'      # user name and password in NETRC
WORKING_DIR = '/content/drive/MyDrive/InSAR-DOH/'  # existing directory within GoogleDrive
HYP3_NODE   =  'urs.earthdata.nasa.gov'
JOB_NAME    = 'DES_CMI_PYO_2021'    #  job name as submitted to ASF Hyp3 INSAR_GAMMA JOB
DEBUG_PROD  = -1 #  -1 download all products, no debug ;  1,2,3,.... few trials 
#####################################################################

import os, re
from pathlib import Path
import pandas as pd
import netrc
import urllib.request

from hyp3_sdk import HyP3

def FindJobs( JOB_TYPE='INSAR_GAMMA' ):
    batch = hyp3.find_jobs()
    batch = hyp3.refresh(batch)
    df_job = None
    for job in batch.jobs:
        job1 =  job.to_dict().copy()
        if job1['job_type']==JOB_TYPE:
            job2 =  job1.pop('job_parameters',None).copy()
            job3 = {**job1,**job2}
            df = pd.DataFrame.from_dict( job3, orient='index' ).transpose()
            #print( df)
            if df_job is None:
                df_job = df
            else:
                df_job = pd.concat( [ df_job, df ], ignore_index=True )
            del df
        else:
            print( 'skipping ....', job1['job_type'] )
    return df_job
#######################################################################
netrc = netrc.netrc(NETRC_CRED)
USER,_,PASSWD = netrc.authenticators( HYP3_NODE )
hyp3 = HyP3(username=USER, password=PASSWD)
# print(hyp3.my_info())

project_path = WORKING_DIR
folder_out_path = os.path.join(project_path, JOB_NAME )
os.makedirs(folder_out_path, exist_ok=True)

df_job = FindJobs()
df_job = df_job[(df_job['name']==JOB_NAME)&(df_job.status_code=='SUCCEEDED') ]
for idx, row in df_job.iterrows():
    if DEBUG_PROD == 0: break
    url = row['files'][0]['url']
    fn = row['files'][0]['filename']
    outpath = os.path.join(folder_out_path, fn)
    if os.path.exists(outpath) or os.path.exists(outpath.split('.zip')[0]):
        print('[{}/{}] {} already exists, skip downloading.'.format(DEBUG_PROD, len(df_job), outpath ) )
    else:
        try:     
          urllib.request.urlretrieve(url, outpath)
          print('[{}/{}] {}... downloaded.'.format(DEBUG_PROD, len(df_job), outpath))
        except:
          print('***ERROR*** retrieving ...', url )
    DEBUG_PROD -= 1

print('*** end job_download() ***')

Overwriting job_download.py


**5. Execute Job_Dowload with automatic resume ....**

In [ ]:
!source activate InSAR; python job_download.py

skipping .... RTC_GAMMA
[-1/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211025T231618_20211106T231618_VVP012_INT80_G_weF_0CF0.zip... downloaded.
[-2/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211106T231618_20211212T231617_VVP036_INT80_G_weF_5CB0.zip... downloaded.
[-3/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211106T231618_20211130T231617_VVP024_INT80_G_weF_19B7.zip... downloaded.
[-4/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211130T231617_20220105T231616_VVR036_INT80_G_weF_2A08.zip... downloaded.
[-5/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211013T231618_20211212T231617_VVP060_INT80_G_weF_7B6E.zip... downloaded.
[-6/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211212T231617_20220105T231616_VVR024_INT80_G_weF_BB29.zip... downloaded.
[-7/117] /content/drive/MyDrive/InSAR-DOH/DES_CMI_PYO_2021/S1AA_20211118T231618_20211130T231617_VVP012_INT80_G_weF_5451.zip... downloaded.
[-8